In [ ]:
library(sicegar)
library(dplyr)
library(tidyr)
library(readr)
library(purrr)
library(here)
library(lubridate)
library(ggplot2)
library(scales)


Data Import and Preparation

In [ ]:
pollen_daily <- read_delim(paste0(here(), "/data/dwh/pollen_dwh_daily.txt"), delim = " ", skip = 17, trim_ws = TRUE)
pollen_daily %<>% 
    mutate(across(is.numeric, ~ if_else(. < 0, NA_real_, .)),
    # datetime = ymd_hm(paste0(YYYY, "-", sprintf("%02d", MM), "-", sprintf("%02d", DD), " ", sprintf("%02d", HH), ":", sprintf("%02d", mm))),
    monthday = paste0(sprintf("%02d", MM), sprintf("%02d", DD))) %>%
    select(-(YYYY:mm)) %>%
    setNames(tolower(names(.))) %>%
    pivot_longer(plo:pcf, names_to = "station", values_to = "conc") %>%
    group_by(parameter, station, monthday) %>%
    summarise(conc = mean(conc, na.rm = TRUE)) %>%
    ungroup() %>%
    filter(monthday != "0229") %>%
    mutate(datetime = ymd(paste0("2001", monthday))) %>%
    split(.$parameter)
pollen_daily




In [ ]:
pollen_daily_sum <- map(pollen_daily, ~.x %>%
    group_by(parameter, monthday) %>%
    summarise(conc = sum(conc, na.rm = TRUE)) %>%
    ungroup() %>%
    mutate(time = 1:365,
    intensity = rescale(conc)) %>%
    select(time, intensity))
pollen_daily_sum

Timeseries Plots

In [ ]:
map(pollen_daily, ~
    ggplot(.x, aes(x = datetime, y = conc, col = station, fill = station)) +
    geom_col() +
    ggtitle(paste("Average Daily", unique(.x$parameter), "Pollen Concentrations for the years 2000-2020")) +
    scale_x_date(date_labels = "%b", date_breaks = "1 month"))

In [ ]:
normalizedInput <- normalizeData(dataInput = pollen_daily_sum[[1]], 
                                                dataInputName = "doubleSigmoidalSample")
# Fit double-sigmoidal model
doubleSigmoidalModel <- multipleFitFunction(dataInput = normalizedInput,
                                                model = "doublesigmoidal",
                                                n_runs_min = 20,
                                                n_runs_max = 500,
                                                showDetails = FALSE)                                                
# Calculate additional parameters
doubleSigmoidalModel <- parameterCalculation(doubleSigmoidalModel)

# Plotting the model
figureModelCurves(dataInput = normalizedInput,
doubleSigmoidalFitVector = doubleSigmoidalModel,
showParameterRelatedLines = TRUE)

In [ ]:
params <- parameterCalculation(doubleSigmoidalModel)
params %<>% 
    mutate(across(everything(), as.character)) %>%
    pivot_longer(everything())
bind_cols(params %>% slice(1:35),
          params %>% slice(36:nrow(params)))